In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
from math_rag.infrastructure.containers import InfrastructureContainer


RESET = True

# containers
infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()
infrastructure_container.wire(modules=[__name__])

application_container = infrastructure_container.application_container()
application_container.init_resources()
application_container.wire(modules=[__name__])

# seed
for object_seeder in infrastructure_container.object_seeders():
    object_seeder.seed(reset=RESET)

for document_seeder in infrastructure_container.document_seeders():
    await document_seeder.seed(reset=RESET)

# index
for document_indexer in infrastructure_container.document_indexers():
    await document_indexer.index(reset=RESET)

2025-06-04 08:37:27,596 - INFO - PyTorch version 2.6.0 available.


In [ ]:
from pathlib import Path


google_drive_repository = infrastructure_container.google_drive_repository()

file_id = google_drive_repository.get_file_id(
    Path('ml/lectures/L07-LogisticRegression2/2024_08_10_2174b40686820b4cb591g.tex')
)

if not file_id:
    raise ValueError()

file_content = google_drive_repository.get_file_by_id(file_id)

In [ ]:
latex = file_content.getvalue().decode()
print(latex)

In [ ]:
latex_parser_service = infrastructure_container.latex_parser_service()
latex_node_walker_service = infrastructure_container.latex_node_walker_service()

In [34]:
text = latex_to_plain_text(latex)
print(text)

In [ ]:
# usage: template.format(**your_dynamic_values)

In [ ]:
from math_rag.core.models import MathExpression


index_id = ...
math_article_id = ...

latex = ...
katex = ...
position = ...
is_inline = ...

math_expression = MathExpression(
    math_article_id=...,
    math_expression_dataset_id=None,
    index_id=index_id,
    latex=latex,
    katex=katex,
    position=position,
    is_inline=is_inline,
)

In [ ]:
import re


text = 'This is [math_placeholder] and another [math_placeholder] here.'

pattern = r'\[math_placeholder\]'
positions = [match.start() for match in re.finditer(pattern, text)]

print(positions)  # Output: [8, 33]

[8, 39]


In [ ]:
def chunk(
    text: str, positions: list[int], lengths: list[int], max_window_size: int = 200
) -> list[str]:
    n = len(positions)
    chunks: list[str] = []

    # 1) build the first window
    # if the very first entity is by itself too long, it can never be chunked
    first_len = lengths[0]

    if first_len > max_window_size:
        raise ValueError(
            f'Entity at position {positions[0]} '
            f'has length {first_len}, which exceeds '
            f'the maximum window size of {max_window_size} characters'
        )

    start_index = 0
    end_index = 0

    # greedily expand until adding the next entity would overflow
    while end_index < n:
        entity_end = positions[end_index] + lengths[end_index]

        if entity_end <= max_window_size:
            end_index += 1
        else:
            break

    # Emit the first chunk
    first_end = positions[end_index - 1] + lengths[end_index - 1]
    chunks.append(text[:first_end])

    # 2) slide the window over each subsequent entity
    for current_index in range(end_index, n):
        new_start = positions[current_index]
        last_end = positions[current_index - 1] + lengths[current_index - 1]

        # gap check
        gap = new_start - last_end

        if gap > max_window_size:
            raise ValueError(
                f'Gap of {gap} characters between entity ending at {last_end} '
                f'and next entity at {new_start} exceeds '
                f'the maximum window size of {max_window_size} characters'
            )

        # slide off old entities until the new one fits
        while start_index < current_index:
            window_start = positions[start_index] if start_index > 0 else 0
            candidate_end = new_start + lengths[current_index]

            if candidate_end - window_start <= max_window_size:
                break
            start_index += 1

        window_start = positions[start_index] if start_index > 0 else 0
        chunks.append(text[window_start : new_start + lengths[current_index]])

    return chunks